In [145]:
import time 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
#options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

In [146]:
def extract_metadata_from(a_dom_element):
    time.sleep(3)
    a_dom_element.click()
    
    tag_labels = [
        '江戸買物ID', '商人名', '居所（歴史地名大系）', '江戸マップID',
        '居所（原本表記）', '職種（原本表記）', '仲間'
    ]
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    metadata = { tag_label: '' for tag_label in tag_labels }
    for tag_dom in soup.select('div#detail_facet a'):
        text = tag_dom.text.strip()
        for tag_label in tag_labels:
            if text.startswith(tag_label):
                label = text.replace(tag_label + '：', '')
                if label:
                    metadata[tag_label] = label

    button_dom = driver.find_element(By.CSS_SELECTOR, 'button#detail_close')
    button_dom.click()
    return metadata
                

In [147]:
def go_next_list():
    try:
        next_dom = driver.find_element(By.CSS_SELECTOR, 'div#thumbnails_nav2 li a.next')
        next_dom.click()
        time.sleep(6)
        return True
    except:
        return False
                

In [148]:
def save_metadata(metadata):
    with open('edo.tsv', 'a+') as f:
        row = "\t".join(metadata.values()) + '\n'          
        f.write(row)

In [149]:
base_url = 'https://codh.rois.ac.jp/edo-shops/iiif-curation-finder/'

path = '?select=canvas&where_metadata_label=出典&where_metadata_value=江戸買物独案内・問屋の部&lang=ja'

driver.get(base_url + path)
time.sleep(3)

metadata_list = []
   
while True:
    for a_dom in driver.find_elements(By.CSS_SELECTOR, 'ul#thumbnails_container li a.thumbnail'):
        metadata = extract_metadata_from(a_dom)
        metadata_list.append(metadata)
        print(len(metadata_list))
        save_metadata(metadata)
    else:
        is_continuable = go_next_list()
        if not is_continuable:
            break

metadata_list

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 